<a href="https://colab.research.google.com/github/Tasneem98AI/final-rnn/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np

sequence = "Tasneem love ice cream"
words = sequence.split()
input_words = words[:3]  # ["Tasneem", "love", "ice"]
target_word = words[3]   # "cream"

vocab = sorted(list(set(words)))
word_to_idx = {w: i for i, w in enumerate(vocab)}
idx_to_word = {i: w for i, w in enumerate(vocab)}
vocab_size = len(vocab)

hidden_size = 64
learning_rate = 0.1
n_epochs = 500

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01
Why = np.random.randn(vocab_size, hidden_size) * 0.01
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def forward(inputs, targets, hprev):
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0

    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))
        xs[t][inputs[t]] = 1

        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        ps[t] = softmax(ys[t])

        if targets is not None and t < len(targets):
            loss += -np.log(ps[t][targets[t], 0])

    return loss, ps, hs, xs

def backward(inputs, targets, ps, hs, xs):
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])

    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        if targets is not None and t < len(targets):
            dy[targets[t]] -= 1

        dWhy += np.dot(dy, hs[t].T)
        dby += dy

        dh = np.dot(Why.T, dy) + dhnext
        dhraw = (1 - hs[t] * hs[t]) * dh

        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)

    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)

    return dWxh, dWhh, dWhy, dbh, dby

inputs = [word_to_idx[w] for w in input_words]
targets = [word_to_idx[target_word]] * len(inputs)  # نفس الطول

hprev = np.zeros((hidden_size, 1))

for epoch in range(n_epochs):
    loss, ps, hs, xs = forward(inputs, targets, hprev)
    dWxh, dWhh, dWhy, dbh, dby = backward(inputs, targets, ps, hs, xs)

    for param, dparam in zip([Wxh, Whh, Why, bh, by], [dWxh, dWhh, dWhy, dbh, dby]):
        param -= learning_rate * dparam

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss}")

def predict_next_word(input_words):
    h = np.zeros((hidden_size, 1))
    input_indices = [word_to_idx[w] for w in input_words]

    for t in range(len(input_indices)):
        x = np.zeros((vocab_size, 1))
        x[input_indices[t]] = 1
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)

    y = np.dot(Why, h) + by
    p = softmax(y)
    predicted_idx = np.argmax(p)
    return idx_to_word[predicted_idx]

print("\n--- Test Model  ---")
test_input = ["Tasneem", "love", "ice"]
predicted = predict_next_word(test_input)
print(f"Inputs: {test_input}")
print(f"Predicted Word : {predicted}")

Epoch 0, Loss: 4.157860456047195
Epoch 50, Loss: 0.056301297955847235
Epoch 100, Loss: 0.01574145358277565
Epoch 150, Loss: 0.00843400875732469
Epoch 200, Loss: 0.005621621004816962
Epoch 250, Loss: 0.004167549014079756
Epoch 300, Loss: 0.0032888782040947735
Epoch 350, Loss: 0.002704189863532171
Epoch 400, Loss: 0.0022888063301218023
Epoch 450, Loss: 0.0019793942307804596

--- Test Model  ---
Inputs: ['Tasneem', 'love', 'ice']
Predicted Word : cream
